In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

# unpack the pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_dir = '/content/drive/MyDrive/FALL_2022_SEMESTER/COMS_4995_Applied_Deep_Learning/Course_Project/music_dataset/'
pickle_file_path = os.path.join(dataset_dir, 'spec_cent_pic.pickle')

In [3]:
# unpack the pickle file
pickle_in = open(pickle_file_path, "rb")
test_data = pickle.load(pickle_in)

In [4]:
# map each raga str to an int
raga_to_int = {
    'Sankarabaranam': 0,
    'Reethigowla': 1,
    'Kalyani': 2,
    'Shanmukhapriya': 3,
    'SindhuBhairavi': 4,
    'Sri': 5,
    'Surutti': 6,
    'Thodi': 7,
    'PoorviKalyani': 8,
    'Mohanam': 9,
    'Madhyamavathi': 10,
    'Kharaharapriya': 11,
    'Keeravani': 12,
    'Kapi': 13,
    'Kambhodhi': 14,
    'Kamas': 15, 
    'Jaganmohini': 16,
    'Hamsadhwani': 17,
    'Bilahari': 18,
    'Bhairavi': 19
}


In [5]:
# OK so now just create a random network and see how it does
# split up into test and train data
X = []
y = []
for val in test_data:
  X.append(val[0])
  y.append(raga_to_int[val[1]])

del test_data




In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
X_train =np.array(X_train)
X_test =np.array(X_test)
y_train =np.array(y_train)
y_test =np.array(y_test)

In [7]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (1173, 1, 2584)
X_test shape: (391, 1, 2584)
y_train shape: (1173,)
y_test shape: (391,)


In [22]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, 2584)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(20)
])

In [23]:
predictions = model(X_train[:1]).numpy()
predictions

array([[ 6914.644  , -3449.583  ,   291.38647,  8156.06   , -4848.0117 ,
          585.16516, -1488.3359 , -5509.9917 ,  -182.37665,  7829.058  ,
         3679.8    ,  3468.6067 ,  1519.7236 , -2819.7761 , -1799.6116 ,
         1316.9832 , -4722.2812 ,  1444.2745 ,  -419.9713 ,  -687.8385 ]],
      dtype=float32)

In [24]:
tf.nn.softmax(predictions).numpy()


array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [25]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [26]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=30)


Epoch 1/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8869 - accuracy: 0.2285
Epoch 2/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8699 - accuracy: 0.2285
Epoch 3/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8538 - accuracy: 0.2285
Epoch 4/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8381 - accuracy: 0.2285
Epoch 5/30
37/37 [==============================] - 0s 5ms/step - loss: 2.8232 - accuracy: 0.2285
Epoch 6/30
37/37 [==============================] - 0s 8ms/step - loss: 2.8089 - accuracy: 0.2285
Epoch 7/30
37/37 [==============================] - 0s 8ms/step - loss: 2.7953 - accuracy: 0.2285
Epoch 8/30
37/37 [==============================] - 0s 8ms/step - loss: 2.7824 - accuracy: 0.2285
Epoch 9/30
37/37 [==============================] - 0s 8ms/step - loss: 2.7699 - accuracy: 0.2285
Epoch 10/30
37/37 [==============================] - 0s 8ms/step - loss: 2.7581 - accuracy: 0.2285
Epoch 11/30
37/37 [

In [32]:
# might have started to overfit on trarin accuracy after 1 epoch???
# try testing it 
model.evaluate(X_test,  y_test, verbose=2)


13/13 - 0s - loss: 2.6655 - accuracy: 0.2123 - 72ms/epoch - 6ms/step


[2.665489912033081, 0.21227622032165527]

In [ ]:
# ok so now, get the actual features of the dataset (Spectral centroid, spectral bandwidth, mel-spectrogram, Mel freq cepstral coeff, chroma)
# Get those into a pickle file, and see how it performs on the same neural network. If accuracy is low, then either modify network or add more data samples